<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/drive/14NMlKInqaI0sP9MqlPaCN1we5pRnWmX3" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelpandas/blob/main/notebooks/annotations.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# _**Creating Data Rows with Annotations with LabelPandas**_

## _**Documentation**_

### **Data Rows**
_____________________

**Requirements:**

- A `row_data` column - This column must be URLs that point to the asset to-be-uploaded

- Either a `dataset_id` column or an input argument for `dataset_id`
  - If uploading to multiple datasets, provide a `dataset_id` column 
  - If uploading to one dataset, provide a `dataset_id` input argument
    - _This can still be a column if it's already in your CSV file_

**Recommended:**
- A `global_key` column
  - This column contains unique identifiers for your data rows
  - If none is provided, will default to your `row_data` column
- An `external_id` column
  - This column contains non-unique identifiers for your data rows
  - If none is provided, will default to your `global_key` column  

**Optional:**
- A `project_id` columm or an input argument for `project_id`
  - If batching to multiple projects, provide a `project_id` column
  - If batching to one project, provide a `project_id` input argument
    - _This can still be a column if it's already in your CSV file_

### **Annotations**
_____________________

*Note:*
*There must also be a `project_id` column, or an input argument for `project_id` when using LabelPandas to upload annotations*

*There must also be an `upload_method` provided when using LabelPandas*
  - *`upload_method` must be one of the following:*
    - *`"mal"` (uploads annotations as pre-labels)*
    - *`"import"` (uploads annotations as submitted labels)*


- For annotations, the column name must be `annotation` + `divider` + `annotation_type` + `divider` + `top_level_feature_name`
  - Example: `annotation///bbox///bbox_tool_name` where, in this case, the bounding tool name in your Labelbox ontology is "bbox_tool_name"
  - `annotation_type` must be one of the following:
    - `bbox`, `polygon`, `point`, `mask`, `line`, `named-entity`, `radio`, `checklist`, `text`
- Values for annotations must correspond with the following, per annotation type:

_____________________
_____________________

**Row-Level Formats for Tool Annotations**
- `bbox` (this example is two annotations)
```
[
        [[top, left, height, width], [nested_classification_name_paths]], 
        [[top, left, height, width], [nested_classification_name_paths]]
]
```
- `polygon` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `line` (this example is two annotations)
```
[
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]], 
        [[(x, y), (x, y),...(x, y)], [nested_classification_name_paths]]
]
```
- `point` (this example is two annotations)
```
[
        [[x, y], [nested_classification_name_paths]], 
        [[x, y], [nested_classification_name_paths]]
]
```
- `mask` (this example is two annotations)
```
[
        [[URL, colorRGB], [nested_classification_name_paths]], 
        [[URL, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[numpy_array, colorRGB], [nested_classification_name_paths]], 
        [[numpy_array, colorRGB], [nested_classification_name_paths]]
]
                      OR
[
        [[png_bytes, None], [nested_classification_name_paths]], 
        [[png_bytes, None], [nested_classification_name_paths]]
]
```
- `named-entity` (this example is two annotations)
```
[
        [[start, end], [nested_classification_name_paths]], 
        [[start, end], [nested_classification_name_paths]]
]
```

**Row-Level Formats for Classification Annotations**
- `radio`, `checklist` and `text`
```
[[answer_name_paths]]
```
  - Note: the last string in a text name path is the text answer value itself
_____________________
_____________________  

## _**Code**_

Install LabelPandas

In [ ]:
!pip install labelpandas --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import labelpandas as lp
import pandas as pd
# Imported to create an example ontology - not required for typical runs of LabelPandas
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option

In [ ]:
csv_path = "https://raw.githubusercontent.com/Labelbox/labelpandas/main/datasets/annotations.csv" # Path to your CSV file
api_key = "" # Labelbox API Key

Load a CSV

In [ ]:
df = pd.read_csv(csv_path)
df.head()

,global_key,row_data,external_id,annotation///bbox///sample_bounding_box,annotation///bbox///sample_nested_bounding_box,annotation///polygon///sample_polygon,annotation///polygon///sample_nested_polygon,annotation///mask///sample_segmentation_mask,annotation///point///sample_point,annotation///line///sample_polyline,annotation///checklist///sample_checklist_question,annotation///radio///sample_nested_radio_question,annotation///radio///sample_radio_question,annotation///text///sample_free_text_question
0,labelpandas-annotations-test-gCbn5IeZtE92OaUby...,https://labelbox.s3-us-west-2.amazonaws.com/da...,gCbn5IeZtE92OaUbyl1ZjQ.jpg,"[[[1853, 191, 213, 304], []], [[1828, 749, 154...","[[[1813, 1066, 259, 285], ['sample_tool_sub_te...","[[[[3363.98, 1180.19], [3205.616, 1349.865], [...","[[[[1341.067, 2550.793], [1412.708, 2545.137],...",[[['iVBORw0KGgoAAAANSUhEUgAAD8AAAAvQCAAAAADlnp...,"[[[1936.818, 2509.317], []], [[732.12, 2473.49...","[[[[1416.479, 1962.584], [2768.229, 2235.95], ...",[['sample_checklist_answer_1']],[['sample_branch_radio_answer_1///sample_sub_r...,[['sample_radio_answer_1']],NaN
1,labelpandas-annotations-test-1MnLIosQZmXH3T-iU...,https://labelbox.s3-us-west-2.amazonaws.com/da...,1MnLIosQZmXH3T-iU-4mtQ.jpg,"[[[1463, 2155, 125, 200], []]]","[[[1362, 938, 237, 198], []]]","[[[[19.327, 2805.486], [701.995, 2515.586], [1...","[[[[1902.743, 1460.723], [1897.132, 1543.017],...",[[['iVBORw0KGgoAAAANSUhEUgAAD6AAAAu4CAAAAADQ3h...,"[[[430.798, 1477.556], []], [[254.988, 1451.37...","[[[[9.975, 1548.628], [797.382, 1658.978], [12...","[['sample_checklist_answer_1', 'sample_checkli...",[['sample_branch_radio_answer_1']],[['sample_radio_answer_2']],[['Sample text answer']]
2,labelpandas-annotations-test-qm4W6ktKCGR22n21A...,https://labelbox.s3-us-west-2.amazonaws.com/da...,qm4W6ktKCGR22n21A3o_0A.jpg,"[[[1315, 2003, 147, 350], []], [[1340, 913, 12...","[[[999, 25, 146, 350], ['sample_tool_sub_text_...","[[[[2000.574, 856.19], [2023.466, 944.327], [2...","[[[[1775.08, 967.219], [1759.055, 1003.848], [...",[[['iVBORw0KGgoAAAANSUhEUgAADMAAAAcsCAAAAADZjE...,"[[[552.606, 554.005], []], [[2016.599, 358.272...","[[[[19.204, 489.905], [558.329, 811.549], [955...",[['sample_checklist_answer_1']],[['sample_leaf_radio_answer_2']],[['sample_radio_answer_1']],NaN


Create a Project and Dataset (for demonstration purposes only)

In [ ]:
client = lp.Client(lb_api_key=api_key)

In [ ]:
dataset = client.lb_client.create_dataset(name="LabelPandas-annotations")
project = client.lb_client.create_project(name="LabelPandas-annotations")

In [ ]:
ontology_builder = OntologyBuilder(
    classifications=[ 
        Classification( # Radio classification
            class_type=Classification.Type.RADIO, instructions="sample_radio_question", 
            options=[Option(value="sample_radio_answer_1"), Option(value="sample_radio_answer_2")]
        ),
        Classification( # Checklist classification
            class_type=Classification.Type.CHECKLIST, instructions="sample_checklist_question", 
            options=[Option(value="sample_checklist_answer_1"), Option(value="sample_checklist_answer_2")]
        ), 
        Classification( # Text classification
            class_type=Classification.Type.TEXT, instructions="sample_free_text_question"
        ),
        Classification( # Radio classification where one answer has a nested radio classification
            class_type=Classification.Type.RADIO, instructions="sample_nested_radio_question",
            options=[
                Option(
                    value="sample_branch_radio_answer_1", 
                    options=[
                        Classification(
                            class_type=Classification.Type.RADIO, instructions="sample_sub_radio_question", 
                            options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                        )
                    ]
                ), 
                Option(value="sample_leaf_radio_answer_2")
            ]
        )
    ],
    tools=[ # List of Tool objects
        Tool( # Bounding Box tool
            tool=Tool.Type.BBOX, name="sample_bounding_box"), 
        Tool( # Bounding Box tool with a nested text classification
            tool=Tool.Type.BBOX,  name="sample_nested_bounding_box",
            classifications=[
                Classification(class_type=Classification.Type.TEXT, instructions="sample_tool_sub_text_question"),]
        ),
        Tool( # Polygon tool
            tool=Tool.Type.POLYGON, name="sample_polygon"
        ),
        Tool( # Polygon tool with a nested radio classification
            tool=Tool.Type.POLYGON, name="sample_nested_polygon",
            classifications=[
                Classification(
                    class_type=Classification.Type.TEXT, instructions="sample_tool_sub_radio_question",
                    options=[Option("sample_sub_radio_answer_1"), Option("sample_sub_radio_answer_2")]
                ),
            ]            
        ),        
        Tool( # Segmentation mask tool given the name "mask"
            tool=Tool.Type.SEGMENTATION, name="sample_segmentation_mask"
        ),
 	      Tool( # Point tool given the name "point"
            tool=Tool.Type.POINT, name="sample_point"
        ), 
        Tool( # Polyline tool given the name "line"
            tool=Tool.Type.LINE, name="sample_polyline"
        )
    ]
)

ontology = client.lb_client.create_ontology("LabelPandas-annotations", ontology_builder.asdict())

project.setup_editor(ontology)

/usr/local/lib/python3.8/dist-packages/labelbox/schema/ontology.py:159: UserWarning: When creating the Classification feature, please use “name” for the classification schema name, which will be used when creating annotation payload for Model-Assisted Labeling Import and Label Import. “instructions” is no longer supported to specify classification schema name.
  warnings.warn(msg)


Upload to Labelbox

In [ ]:
results = client.create_data_rows_from_table(
    table = df,
    dataset_id = dataset.uid,
    project_id = project.uid,
    upload_method = "import", # Must be either "import" or "mal"
    skip_duplicates = False, # If True, will skip data rows where a global key is already in use
    mask_method = "png", # Input masks must be either "png", "url", or "array"
    verbose = True, # If True, prints information about code execution
)

Creating upload list - 3 rows in Pandas DataFrame
Beginning data row upload for dataset ID cleesmidf7teh070l571h1tnp: uploading 3 data rows
Batch #1: 3 data rows
Success: Upload batch number 1 successful
Upload complete - all data rows uploaded
Sending 3 data rows to project with ID cleesmjip2cg107yz3hyzf5hl
All data rows have been batched to the specified project(s)
Uploading annotations as submitted labels (Label Import)
Uploading 51 annotations for 3 data rows to project with ID cleesmjip2cg107yz3hyzf5hl
Success: upload batch number 1 complete
